In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn import metrics
import tqdm
import pickle as pkl

In [ ]:
df = pd.read_csv('data/qm8.csv')

In [ ]:
df.head()

In [ ]:
tqdm.tqdm.pandas()

In [ ]:
def smile_is_3d(smile):
    mol = Chem.MolFromSmiles(smile)
    mol = Chem.AddHs(mol)
    AllChem.EmbedMolecule(mol)
    
    is_3d = False
    for c in mol.GetConformers():
        is_3d |= c.Is3D()
        
    return is_3d

In [ ]:
def smile_features(smile):
#     print(smile)
    mol = Chem.MolFromSmiles(smile)
    mol = Chem.AddHs(mol)
    
    is_3D = False    
    atomic_nums = [a.GetAtomicNum() for a in mol.GetAtoms()]
    
    euclid_D = Chem.rdDistGeom.GetMoleculeBoundsMatrix(mol)
    graph_D = Chem.GetDistanceMatrix(mol)
    return (atomic_nums, euclid_D, graph_D, is_3D)

In [ ]:
mol = Chem.MolFromSmiles('[H]C([H])([H])C12C([H])([H])N(C1([H])[H])C2([H])[H]')
mol = Chem.AddHs(mol)
AllChem.EmbedMolecule(mol)
# AllChem.MMFFOptimizeMolecule(mol)

In [ ]:
mol.GetConformer(0).Is3D()

In [ ]:
Chem.GetDistanceMatrix(mol)

In [ ]:
import numpy as np

In [ ]:
sample_df = df

In [ ]:
res = df.smiles.head(10).progress_apply(smile_features).to_list()

In [ ]:
sample_df.loc[:, ['Z', 'euclid_D', 'graph_D', 'is_3D']] = sample_df.smiles.progress_apply(smile_features).to_list()

# Test loading dataset

In [ ]:
import data
from torch.utils.data import Dataset, DataLoader

In [ ]:
ds = data.QM8Dataset('data/rdkit_bound.json', 'E1-CC2', 5,
                     mu_max=-1, nrows=1, dist_method='graph')

In [ ]:
ds[0][1].squeeze()

In [ ]:
ds[0][1].squeeze()

In [ ]:
dl = DataLoader(ds, 10)

In [ ]:
Zs, Ds, sizes, target = next(iter(dl))

In [ ]:
Zs.shape, Ds.shape, sizes.shape, target.shape

In [ ]:
df = pd.read_json('data/rdkit_bound.json', 'records', lines=True)

In [ ]:
df.Z.apply(max).max()

In [ ]:
df.is_3D.value_counts()